# Twitter Sentiment Analysis for the word Euro

Connect Spark to Mongo DB

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars "/usr/local/spark/jars/mongo-spark-connector_2.12-3.0.2.jar,/usr/local/spark/jars/mongo-java-driver-3.12.9.jar" pyspark-shell'

Note the inferschema is set to false, this makes spark read the entire database and not infer the values of fields from the first set of fields

In [ ]:
from pyspark.sql import SparkSession
# fix read bug, basically turn off sampling
spark = SparkSession.builder.appName("TwitterMongo") \
.config("spark.mongodb.input.database", "mongodb://localhost:27017/twitter") \
.config("spark.mongodb.input.uri", "mongodb://localhost:27017/twitter.tweets") \
.config("spark.mongodb.read.sql.inferSchema.mapTypes.enabled", "FALSE") \
.config("spark.mongodb.output.uri","mongodb://localhost:27017/twitter.tweets").getOrCreate()

## Create the Session

And load all of the Twitter data in MongoDB

Print out the twitter tweet schema

In [ ]:
# create a spark session
spark = SparkSession \
.builder \
.master("local") \
.appName("ABC") \
.config("spark.driver.memory", "15g") \
.config("spark.mongodb.read.connection.uri", "mongodb://localhost:27017/twitter") \
.config("spark.mongodb.write.connection.uri", "mongodb://localhost:27017/twitter") \
.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:2.12-3.0.2') \
.getOrCreate()
# read data from mongodb collection "questions" into a dataframe "df"
df = spark.read \
.format("com.mongodb.spark.sql.DefaultSource") \
.option("uri", "mongodb://localhost:27017/twitter") \
.option("database", "twitter") \
.option("collection", "tweets") \
.load()
df.printSchema()

In [ ]:
df.show()

Create a spark object of the tweets held in the mongo db 

It is easier to use SQL statements and Pyspark to clean the data rather than writing queries in MongoDB

In [ ]:
df.createOrReplaceTempView("tweets")

How many tweets in the DB all together

In [ ]:
df = spark.sql("SELECT DISTINCT id FROM tweets")
df.count()

How many tweets by language 


In [ ]:
#pip install plotly
import pyspark.pandas as ps
import plotly
dfLang = spark.sql("SELECT DISTINCT lang, CAST(count(id) AS INT) as TweetCount FROM tweets GROUP BY lang \
                   ORDER BY TweetCount DESC LIMIT 10")
dfLang.show()

In [ ]:
tempdf = ps.DataFrame(dfLang)

tempdf.plot(kind='bar', x='lang', y='TweetCount')

How many tweets by location

In [ ]:
dfLoc = spark.sql("SELECT DISTINCT user.location AS Location, CAST(count(id) AS INT) as TweetCount FROM tweets GROUP BY user.location \
                   ORDER BY TweetCount DESC LIMIT 10")
dfLoc.show()

In [ ]:
tempdf = ps.DataFrame(dfLoc)

tempdf.plot(kind='bar', x='Location', y='TweetCount')

Now limit the dataset to English texts and tweets with the Euro in the text

In [ ]:
dfEnTwt = spark.sql("SELECT * FROM tweets WHERE lang = 'en' AND text LIKE '%euro%'")
dfEnTwt.createOrReplaceTempView("en_tweets")
dfEnTwt.show()

Now how many tweets in the English language dataset

In [ ]:
dfEnTwt.count()

In [ ]:
# select tweet id, geo, lang, quoted_status,quoted_status.geo 
# Having a look at some the data
dfOne = spark.sql("SELECT DISTINCT id,  text, quote_count, reply_count, retweet_count, favorite_count, geo, place, lang, \
                  quoted_status,quoted_status.geo, quoted_status.text, user.name, user.location   \
                  FROM en_tweets")

In [ ]:
dfOne.show()

In [ ]:

dfDay= spark.sql("SELECT DISTINCT CAST(substring(created_at, 27, 4) AS INT) as Year, \
          CAST(from_unixtime(unix_timestamp(substring(created_at, 5, 3), 'MMM'), 'MM') As INT) as Month, \
          CAST(substring(created_at, 9, 2) AS INT) as Day, \
          CAST(count(id) AS INT) as TweetCount \
          FROM en_tweets \
          WHERE lang = 'en' AND text LIKE '%euro%' GROUP BY substring(created_at, 27, 4), \
          substring(created_at, 5, 3), \
          substring(created_at, 9, 2)")
dfDay.createOrReplaceTempView("tweetsByDay")

dfDay = spark.sql("SELECT * FROM tweetsByDay ORDER BY Year, Month, Day")

dfDay.show()


In [ ]:
tempdf = ps.DataFrame(dfDay)

tempdf.plot(kind='bar', x='Day', y='TweetCount')

In [ ]:
dfWeek = spark.sql("SELECT Year, weekofyear(make_date(Year, Month, Day)) as wkofYr , SUM(TweetCount) as TweetCountbyWeek \
                   FROM tweetsByDay \
                   GROUP BY Year, weekofyear(make_date(Year, Month, Day))")

dfWeek.createOrReplaceTempView("tweetsByWeek")
dfWeek.show()

In [ ]:
tempdf = ps.DataFrame(dfWeek)

tempdf.plot(kind='bar', x='wkofYr', y='TweetCountbyWeek')

In [ ]:
# get the tweet count by month
dfMonth = spark.sql("SELECT Year, Month, SUM(TweetCount) as TweetCountbyMonth \
                     FROM tweetsByDay GROUP BY Year, Month")
dfMonth.createOrReplaceTempView("tweetsByMonth")
dfMonth.show()

In [ ]:
tempdf = ps.DataFrame(dfMonth)

tempdf.plot(kind='bar', x='Month', y='TweetCountbyMonth')

### Next up is text clean up 

In [ ]:
#%pip install wordcloud
#%pip install vadersentiment
## sentiment analysis ref: https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/?ref=lbp
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
     
    print("Overall sentiment dictionary is : ", sentiment_dict)
    #print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
    #print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
    #print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")
 
    #print("Sentence Overall Rated As", end = " ")
 
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        print("Positive")
 
    elif sentiment_dict['compound'] <= - 0.05 :
       print("Negative")
 
    else :
        print("Neutral")
    
    return sentiment_dict['compound']

In [ ]:
dfText = spark.sql("SELECT DISTINCT LOWER(text) AS text FROM en_tweets")
dfText.createOrReplaceTempView("text")
dfText.show()


Count the number of words in the tweets 

In [ ]:
from pyspark.sql.functions import * 
from pyspark.sql.types import StringType, ArrayType
dfWord = spark.sql("SELECT explode(split(text, ' ')) AS word FROM text")

dfWord.show()

Count the number of characters including spaces

In [ ]:
dfChar = spark.sql("SELECT len(text) AS char FROM text")
dfChar.show()

In [ ]:

pattern = r'[^a-zA-Z0-9\s]'

dfText = spark.sql("SELECT * FROM text")
dfText = dfText.withColumn("text", regexp_replace('text', pattern, ''))
dfText.take(5)

In [ ]:
dfText.createOrReplaceTempView("text") # this is cleaned txt
dfText = spark.sql("SELECT TRIM(text) AS text FROM text")

dfText.take(5)

In [ ]:
#now build a word cloud
from wordcloud import WordCloud, STOPWORDS
dfText.createOrReplaceTempView("text") # this is the trimmed txt
dfText = spark.sql("SELECT text FROM text")
dfText.show(5)

### Tokenize and Stem the tweets

In [ ]:
#%pip install nltk

In [ ]:
# stackoverflow ref: https://stackoverflow.com/questions/53579444

from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer
from nltk.stem.snowball import SnowballStemmer

In [ ]:
# Tokenize text
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
dfText = tokenizer.transform(dfText).select("text","tokens")

dfText.show(5)

In [ ]:
# Remove stopwords
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered")
dfText = remover.transform(dfText).select("text","filtered")

dfText.show(5)

In [ ]:
# stem the words
stemmer = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
dfText = dfText.withColumn("filtered_stemmed", stemmer_udf("filtered"))
dfText.show(5)

In [ ]:
# Filter out short words
filterShortWords = udf(lambda row: [x for x in row if len(x) >= 4], ArrayType(StringType()))
dfText = dfText.withColumn("filtered_stemmed", filterShortWords("filtered_stemmed"))

dfText.show(5)
